In [1]:
import pickle
import psycopg2
import math
import numpy as np
import io
import re
import sys
sys.path.append('..')
from utils.another import get_ngrams, key_dependent_dict, condition
from collections import Counter

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\persi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
conn = psycopg2.connect(host="localhost", database="postgres", user="postgres", password="igsvemina1201")
cur = conn.cursor()

In [3]:
cur.execute('SELECT count(*) from unigram_counter')
vocab_size = cur.fetchone()[0]
cur.execute('SELECT sum(unigram_count) from unigram_counter')
n_unigrams = cur.fetchone()[0]
cur.execute('SELECT sum(bigram_count) from bigram_counter')
n_bigrams = cur.fetchone()[0]
cur.execute('SELECT sum(trigram_count) from trigram_counter')
n_trigrams = cur.fetchone()[0]

print(f'vocab size - {vocab_size} \
      \nnumber of ngrams in train corpus:\nungirams - {n_unigrams}\nbigrams - {n_bigrams}\ntrigrams - {n_trigrams}')

vocab size - 2183895       
number of ngrams in train corpus:
ungirams - 829250940
bigrams - 798949912
trigrams - 768648884


In [5]:
test_file_path='../data/test_v2.txt'
test_unigram_counter = Counter()
test_bigram_counter = Counter()
test_trigram_counter = Counter()
with io.open(test_file_path, encoding='utf-8', mode='rt') as test_file:
    for line in test_file:
        comma_pos = line.find(',')
        line = line[comma_pos+1:].rstrip()
        line = line[1:-1] # skip "
        sentence = ["<s>"] + line.lower().split(' ') + ["</s>"]
        test_unigram_counter.update(sentence)
        test_bigram_counter.update(get_ngrams(sentence, 2))
        test_trigram_counter.update(get_ngrams(sentence, 3))
with open('../data/test_unigram_counter.csv', 'w', encoding='utf-8') as unigram_counter_csv, \
    open('../data/test_bigram_counter.csv', 'w', encoding='utf-8') as bigram_counter_csv, \
    open('../data/test_trigram_counter.csv', 'w', encoding='utf-8') as trigram_counter_csv:
    unigram_counter_csv.write('first_gram,unigram_count\n')
    for unigram, count in test_unigram_counter.items():
        unigram_counter_csv.write('{} {}\n'.format(unigram, count))
    bigram_counter_csv.write('first_gram,second_gram,bigram_count\n')
    for bigram, count in test_bigram_counter.items():
        bigram_counter_csv.write('{} {}\n'.format(' '.join(bigram), count))
    trigram_counter_csv.write('first_gram,second_gram,third_gram,trigram_count\n')
    for trigram, count in test_trigram_counter.items():
        trigram_counter_csv.write('{} {}\n'.format(' '.join(trigram), count))
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS test_unigram_counter')
cur.execute('DROP TABLE IF EXISTS test_bigram_counter')
cur.execute('DROP TABLE IF EXISTS test_trigram_counter')
cur.execute('''CREATE TABLE test_unigram_counter( first_gram VARCHAR, unigram_count INT)''')
cur.execute('''CREATE TABLE test_bigram_counter( first_gram VARCHAR, second_gram VARCHAR, bigram_count INT)''')
cur.execute('''CREATE TABLE test_trigram_counter( first_gram VARCHAR, second_gram VARCHAR, \
            third_gram VARCHAR, trigram_count INT)''')
cur.execute("COPY test_unigram_counter FROM 'C:/HSE/3_module/BWI_project/data/test_unigram_counter.csv' DELIMITER ' ' CSV HEADER")
cur.execute("COPY test_bigram_counter FROM 'C:/HSE/3_module/BWI_project/data/test_bigram_counter.csv' DELIMITER ' ' CSV HEADER")
cur.execute("COPY test_trigram_counter FROM 'C:/HSE/3_module/BWI_project/data/test_trigram_counter.csv' DELIMITER ' ' CSV HEADER")
conn.commit()
cur.close()

In [6]:
cur = conn.cursor()
# extract test unigram count from train table
cur.execute("SELECT * FROM unigram_counter as T WHERE T.first_gram in (SELECT first_gram FROM test_unigram_counter)")
train_unigram_counter = dict(cur.fetchall())
cur.execute("SELECT * FROM bigram_counter as T WHERE (T.first_gram,T.second_gram) in (SELECT first_gram,second_gram FROM test_bigram_counter)")
train_bigram_counter = dict(map(lambda x: ((x[0],x[1]),x[2]), cur.fetchall()))
cur.execute("SELECT * FROM trigram_counter as T WHERE (T.first_gram,T.second_gram,T.third_gram) in (SELECT first_gram,second_gram,third_gram FROM test_trigram_counter)")
train_trigram_counter = dict(map(lambda x: (x[:-1],x[-1]), cur.fetchall()))

In [24]:
def encode_line(line):
    comma_pos = line.find(',')
    sentence = line[comma_pos+1:].rstrip()
    sentence = sentence[1:-1] # skip "
    return line[:comma_pos+1], sentence

def decode_sentence(sentence):
    return '"' + sentence + '"'+'\n'

In [105]:
# new Lm
class InterpolationLanguageModel:
    def __init__(self, n=2, lambdas=[0.5,0.5], smoothing='add_one', vocab_size=False):
        assert len(lambdas) == n
        assert sum(lambdas) == 1.0
        assert n==2 or n==3
        self.n = n
        self.lambdas = {(i+1):l for i, l in enumerate(lambdas)}
        self.addone_smoothing = smoothing == 'add_one'
        self.good_tuning = smoothing == 'good_turing'
        if self.addone_smoothing and not vocab_size:
            raise Exception
        self.vocab_size = vocab_size

    def fit(self, train_counters, n_ngrams):  
        ngrams_statistics = {1:{'total':n_ngrams[1], 'counter':train_counters[1]}, \
                                  2:{'total':n_ngrams[2], 'counter':train_counters[2]}, \
                                  3:{'total':n_ngrams[3], 'counter':train_counters[3]}}
        # ADD-ONE SMOOTHING
        if self.addone_smoothing:
            one, v = 1, self.vocab_size
            for n in range(1,4):
                # probabilities for unseen ngrams
                if n == 1:
                    compute_p_unseen = lambda x : (ngrams_statistics[n]['counter'].get(x,0) + one) / \
                                  (ngrams_statistics[n]['total'] + v)
                else:
                    compute_p_unseen = lambda x : (ngrams_statistics[n]['counter'].get(x,0) + one) / \
                                  (ngrams_statistics[n-1]['counter'].get(condition(x),0) + v)
                ngrams_statistics[n]['probs'] = key_dependent_dict(compute_p_unseen)
                # probabilities for known ngrams
                for ngram, count in ngrams_statistics[n]['counter'].items():
                    condition_count = ngrams_statistics[n]['total'] if n==1 else \
                                      ngrams_statistics[n-1]['counter'][condition(ngram)]
                    ngrams_statistics[n]['probs'][ngram] = (count + one) / (condition_count + v)           
        self.ngrams_statistics = ngrams_statistics
            
    def get_prob(self, ngram):
        p = 0
        for n in range(self.n,0,-1):
            p += self.lambdas[n] * self.ngrams_statistics[n]['probs'][ngram]
            ngram = ngram[-1] if n==2 else ngram[1:]     
        return p
            
    def test(self, ngram_counter):
        assert len(list(ngram_counter.keys())[0]) == self.n
        perplexity = 0             
        n_ngrams = sum(ngram_counter.values())
        for ngram, count in ngram_counter.items():
            perplexity += count * math.log(self.get_prob(ngram))
        perplexity = math.pow(2, -1*(perplexity/n_ngrams))        
        return perplexity
    
    def insert_missing_word(self, sentence):
        tokenized_sentence = ["<s>"] + sentence.lower().split() + ["</s>"]
        # slice ngrams under consideration
        slicer = slice(1,-1) if self.n == 2 else slice(0,-2)
        sentence_ngrams = list(get_ngrams(tokenized_sentence, self.n))[slicer]
        minP_ngram_arg = np.argmin([self.get_prob(ngram) for ngram in sentence_ngrams])
        minP_ngram = sentence_ngrams[minP_ngram_arg]
        spaces_inds = [m.start() for m in re.finditer(' ', sentence)]
        space_ind = spaces_inds[minP_ngram_arg]
        context, word = minP_ngram[:-1], minP_ngram[-1]
        candidates = [key[-1] for key in self.ngrams_statistics[self.n]['probs'].keys() if key[:-1] == context]
        argmax = np.argmax([self.get_prob(context+(candidate,))*self.get_prob((candidate,word)) for candidate in candidates])
        missing_word = candidates[argmax]
        outline = sentence[:space_ind+1] + missing_word + sentence[space_ind:]
        return outline
        
    def fill_gaps(self, test_file_path, submission_file_path='submission.txt'):
        with io.open(test_file_path, encoding='utf-8', mode='rt') as infile, \
            io.open(submission_file_path, encoding='utf-8', mode='wt') as outfile:
            outfile.write(infile.readline()+'\n')
            for i, line in enumerate(infile):
#                 if (i+1) % 1000 == 0:
                print(f'working on {i+1} file', end='\r')
                n_line, sentence = encode_line(line)
                sentence = self.insert_missing_word(sentence)
                outline = n_line + decode_sentence(sentence)
                outfile.write(outline)

In [106]:
train_counters = {1:train_unigram_counter, 2:train_bigram_counter, 3:train_trigram_counter}
n_ngrams = {1:n_unigrams, 2:n_bigrams, 3:n_trigrams}
lm = InterpolationLanguageModel(smoothing='add_one',vocab_size=vocab_size)
lm.fit(train_counters, n_ngrams)

In [ ]:
print(f'perplexity = {lm.test(ngram_counter=test_bigram_counter):.2f}')
lm.fill_gaps(test_file_path=test_file_path, submission_file_path='../data/submission.txt')

perplexity = 121.23


# draft

#### smoothing

In [23]:
from sklearn.linear_model import LinearRegression
uni_coc = Counter(sorted(train_unigram_counter.values()))
log_r = np.log10(list(uni_coc.keys())).reshape(-1, 1)
log_Z = np.log10(list(uni_coc.values())).reshape(-1, 1)
reg = LinearRegression().fit(log_r, log_Z)
a, b = reg.intercept_, reg.coef_[0]

In [24]:
a, b

(array([2.55806461]), array([-0.57176488]))

In [197]:
#finds the slope for the best fit line
def findBestFitSlope(x,y):
    m = (( mean(x)*mean(y) - mean(x*y) ) / 
          ( mean(x)** 2 - mean(x**2)))

    return m
      
#finds the intercept for the best fit line
def findBestFitIntercept(x,y,m):
    c = mean(y) - m*mean(x)
    return c

In [ ]:
log_r = np.log10(r).reshape(-1, 1)
log_Z = np.log10(Z).reshape(-1, 1)
reg = LinearRegression().fit(log_r, log_Z)
a, b = reg.intercept_, reg.coef_[0]
S = np.power(10, a+b*np.log(np.arange(1, np.max(r)+2)))

# trash

In [37]:
# new Lm
class InterpolationLanguageModel:
    def __init__(self, lambda1=0.1, lambda2=0.3, lambda3=0.6, smoothing='add_one', vocab_size=False):
        self.lambdas = {1:lambda1, 2:lambda2, 3:lambda3}
        self.addone_smoothing = smoothing == 'add_one'
        self.good_tuning = smoothing == 'good_turing'
        if self.addone_smoothing and not vocab_size:
            raise Exception
        self.vocab_size = vocab_size

    def fit(self, train_counters, n_ngrams):  
        ngrams_statistics = {1:{'total':n_ngrams[1], 'counter':train_counters[1]}, \
                                  2:{'total':n_ngrams[2], 'counter':train_counters[2]}, \
                                  3:{'total':n_ngrams[3], 'counter':train_counters[3]}}
        # ADD-ONE SMOOTHING
        if self.addone_smoothing:
            one, v = 1, self.vocab_size
            for n in range(1,4):
                # probabilities for unseen ngrams
                if n == 1:
                    compute_p_unseen = lambda x : (ngrams_statistics[n]['counter'].get(x,0) + one) / \
                                  (ngrams_statistics[n]['total'] + v)
                else:
                    compute_p_unseen = lambda x : (ngrams_statistics[n]['counter'].get(x,0) + one) / \
                                  (ngrams_statistics[n-1]['counter'].get(condition(x),0) + v)
                ngrams_statistics[n]['probs'] = key_dependent_dict(compute_p_unseen)
                # probabilities for known ngrams
                for ngram, count in ngrams_statistics[n]['counter'].items():
                    condition_count = ngrams_statistics[n]['total'] if n==1 else \
                                      ngrams_statistics[n-1]['counter'][condition(ngram)]
                    ngrams_statistics[n]['probs'][ngram] = (count + one) / (condition_count + v)
        # GOOD TURING
        else:
            for n in ngrams_statistics:
                ngrams_statistics[n]['counter_of_counts'] = Counter(sorted(ngrams_statistics[n]['counter'].values()))
                ngrams_statistics[n]['smoothed_counter'] = defaultdict(int)
                # probabilities for unseen ngrams
                ngrams_statistics[n]['probs'] = defaultdict(lambda: ngrams_statistics[n]['counter_of_counts'] / \
                                                                       ngrams_statistics[n]['total'])
            def adjust_count(count, n):
                return (count + 1) * ngrams_statistics[n]['counter_of_counts'][count+1] / \
                        ngrams_statistics[n]['counter_of_counts'][count]
            # adjuct counts(compute c*) and compute probabilities for known ngrams 
            for n in range(1,4):
                for ngram, count in ngrams_statistics[n]['counter'].items():
                    new_c = adjust_count(count, n)
                    if ngram == 'confirmed':
                        print(ngrams_statistics[n]['counter_of_counts'][count+1], ngrams_statistics[n]['counter_of_counts'][count])
                    ngrams_statistics[n]['smoothed_counter'][ngram] = new_c
                    denominator = ngrams_statistics[n]['total'] if n==1 else \
                                  ngrams_statistics[n-1]['smoothed_counter'][condition(ngram)]
                    if denominator==0: 
                        print('n',n,'\nngram',ngram,'\ncondition',condition(ngram),'\n')
                    ngrams_statistics[n]['probs'][ngram] = new_c / denominator
        self.ngrams_statistics = ngrams_statistics
            
            
    def logprob(self, trigram, alpha1=1, alpha2=1):
        p = 0
        p = self.lambdas[1] * self.ngrams_statistics[1]['probs'][trigram[-1]]
        p += self.lambdas[2] * self.ngrams_statistics[2]['probs'][trigram[1:]]
        p += self.lambdas[3] * self.ngrams_statistics[3]['probs'][trigram]       
        return math.log2(p)
            
    def test(self):
        perplexity = 0             
        n_trigrams = sum(test_trigram_counter.values())
        for trigram, count in test_trigram_counter.items():
            perplexity += count * self.logprob(trigram)
        perplexity = math.pow(2, -1*(perplexity/n_trigrams))     
        
        return perplexity

In [ ]:
# GOOD TURING
        else:
            for n in ngrams_statistics:
                ngrams_statistics[n]['counter_of_counts'] = Counter(ngrams_statistics[n]['counter'].values())
                ngrams_statistics[n]['smoothed_counter'] = defaultdict(int)
                # probabilities for unseen ngrams
                ngrams_statistics[n]['probs'] = defaultdict(lambda: ngrams_statistics[n]['counter_of_counts'] / \
                                                                       ngrams_statistics[n]['total'])
            def adjust_count(count, n):
                return (count + 1) * ngrams_statistics[n]['counter_of_counts'][count+1] / \
                        ngrams_statistics[n]['counter_of_counts'][count]
            # adjuct counts(compute c*) and compute probabilities for known ngrams 
            for n in range(1,4):
                for ngram, count in ngrams_statistics[n]['counter'].items():
                    new_c = adjust_count(count, n)
                    if ngram == 'confirmed':
                        print(ngrams_statistics[n]['counter_of_counts'][count+1], ngrams_statistics[n]['counter_of_counts'][count])
                    ngrams_statistics[n]['smoothed_counter'][ngram] = new_c
                    denominator = ngrams_statistics[n]['total'] if n==1 else \
                                  ngrams_statistics[n-1]['smoothed_counter'][condition(ngram)]
                    if denominator==0: 
                        print('n',n,'\nngram',ngram,'\ncondition',condition(ngram),'\n')
                    ngrams_statistics[n]['probs'][ngram] = new_c / denominator
        self.ngrams_statistics = ngrams_statistics

In [ ]:
# something about backoff smoothing
            # N_c - the count of things we've seen c times
#         else:
#             if trigram in train_trigram_counter:
#                 p = self.lambda1 * self.unigram_probs[trigram[-1]]
#                 p += self.lambda2 * self.bigram_probs[trigram[1:]]
#                 p += self.lambda3 * self.trigram_probs[trigram]
#             elif trigram[1:] in train_bigram_counter:
#                 p = (self.lambda1 * alpha1 + self.lambda2) * self.bigram_probs[trigram[1:]]
#                 p += self.lambda3 * self.trigram_probs[trigram]
#             elif trigram[-1] in train_unigram_counter:
#                 p = (self.lambda1 * alpha2 + self.lambda2 * 0.4 + self.lambda3) * self.trigram_probs[trigram]
#             else:
#                 return 

In [ ]:
# first version of fit
    def fit(self, train_counters, n_unigrams):        
        if self.addone_smoothing:
            one, v = 1, self.vocab_size
            self.unigram_probs = defaultdict(lambda: one / (n_unigrams + v))
            self.bigram_probs = dict()
            self.trigram_probs = dict()
            for unigram in train_unigram_counter:
                self.unigram_probs[unigram] = (train_unigram_counter[unigram] + one) / (n_unigrams + v)
            for bigram in train_bigram_counter:
                condition, _ = bigram
                self.bigram_probs[bigram] = (train_bigram_counter[bigram] + one) / (train_unigram_counter[condition] + v)
            for trigram in train_trigram_counter:
                condition, _ = trigram[:-1], trigram[-1]
                self.trigram_probs[trigram] = (train_trigram_counter[trigram] + one) / (train_bigram_counter[condition] + v)
        elif self.good_tuning:
            counter_of_unigram_counts = Counter(train_unigram_counter.values())
            counter_of_bigram_counts = Counter(train_bigram_counter.values())
            counter_of_trigram_counts = Counter(train_trigram_counter.values())
            counters_of_counts = {1:counter_of_unigram_counts, 2:counter_of_bigram_counts, 3:counter_of_trigram_counts}
            self.smoothed_unigram_counter = dict()
            self.smoothed_bigram_counter = dict()
            self.smoothed_trigram_counter = dict()
            def adjust_count(count, n):
                return (count + 1) * counters_of_counts[n][count+1] / counters_of_counts[n][count]
            # GOOD TURING
            # probabilities for unseen ngrams
            self.unigram_probs = defaultdict(lambda: counters_of_counts[1][1] / n_unigrams)
            self.bigram_probs = defaultdict(lambda: counters_of_counts[2][1] /  n_bigrams)
            self.trigram_probs = defaultdict(lambda: counters_of_counts[3][1] / n_trigrams)
            # adjuct counts(compute c*) and compute ngram probabilities
            for unigram in train_unigram_counter:
                self.smoothed_unigram_counter[unigram] = adjust_count(train_unigram_counter[unigram], 1)
            for unigram, count in self.smoothed_unigram_counter.items():
                self.unigram_probs[unigram] = count  / n_unigrams
            for bigram in train_bigram_counter:
                self.smoothed_bigram_counter[bigram] = adjust_count(train_bigram_counter[bigram], 2)
            for bigram, count in self.smoothed_bigram_counter.items():
                condition, _ = bigram
                self.bigram_probs[bigram] = count / self.smoothed_unigram_counter[condition]
            for trigram in train_trigram_counter:
                self.smoothed_trigram_counter[trigram] = adjust_count(train_trigram_counter[trigram], 3)
            for trigram, count in smoothed_trigram_counter.items():
                condition, _ = trigram[:-1], trigram[-1]
                self.trigram_probs[trigram] = count / self.smoothed_bigram_counter[condition]

In [106]:
class LanguageModel:
    def __init__(self, lambda1=0.1, lambda2=0.3, lambda3=0.6, addone_smoothing=True):
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.lambda3 = lambda3
        self.addone_smoothing = addone_smoothing
        self.good_tuning = not addone_smoothing
    
    def fit(self):        
        if self.addone_smoothing:
            one, v = 1, vocab_size
            self.unigram_probs = defaultdict(lambda: one / (n_unigrams + v))
            self.bigram_probs = defaultdict(lambda: one /  .......)
            self.trigram_probs = defaultdict(lambda: one / .....)
            for unigram in train_unigram_counter:
                self.unigram_probs[unigram] = (train_unigram_counter[unigram] + one) / (n_unigrams + v)
            for bigram in train_bigram_counter:
                condition, _ = bigram
                self.bigram_probs[bigram] = (train_bigram_counter[bigram] + one) / (train_unigram_counter[condition] + v)
            for trigram in train_trigram_counter:
                condition, _ = trigram[:-1], trigram[-1]
                self.trigram_probs[trigram] = (train_trigram_counter[trigram] + one) / (train_bigram_counter[condition] + v)
        elif self.good_tuning:
            counter_of_unigram_counts = Counter(train_unigram_counter.values())
            counter_of_bigram_counts = Counter(train_bigram_counter.values())
            counter_of_trigram_counts = Counter(train_trigram_counter.values())
            counters_of_counts = {1:counter_of_unigram_counts, 2:counter_of_bigram_counts, 3:counter_of_trigram_counts}
            self.smoothed_unigram_counter = defaultdict(lambda:0)
            self.smoothed_bigram_counter = defaultdict(lambda: 0)
            self.smoothed_trigram_counter = defaultdict(lambda: 0)
            self.unigram_probs = defaultdict(lambda: counters_of_counts[1][1] / n_unigrams)
            self.bigram_probs = defaultdict(lambda: counters_of_counts[2][1] /  n_bigrams)
            self.trigram_probs = defaultdict(lambda: counters_of_counts[3][1] / n_trigrams)
            def adjust_count(count, n):
                return (count + 1) * counters_of_counts[n][count+1] / counters_of_counts[n][count]
            # good tuning
            for unigram in train_unigram_counter:
                self.smoothed_unigram_counter[unigram] = adjust_count(train_unigram_counter[unigram], 1)
            for bigram in train_bigram_counter:
                self.smoothed_bigram_counter[bigram] = adjust_count(train_bigram_counter[bigram], 2)
            for trigram in train_trigram_counter:
                self.smoothed_trigram_counter[trigram] = adjust_count(train_trigram_counter[trigram], 3)
            for unigram, count in self.smoothed_unigram_counter.items():
                self.unigram_probs[unigram] = count  / n_unigrams
            for bigram, count in self.smoothed_bigram_counter.items():
                condition, _ = bigram
                self.bigram_probs[bigram] = count / self.smoothed_unigram_counter[condition]
            for trigram, count in smoothed_trigram_counter.items():
                condition, _ = trigram[:-1], trigram[-1]
                self.trigram_probs[trigram] = count / (train_bigram_counter[condition]
                                                       
            
            
            
    def logprob(self, trigram, alpha1=1, alpha2=1):
        p = 0
        if self.addone_smoothing:
            p = self.lambda1 * self.unigram_probs[trigram[-1]]
            p += self.lambda2 * self.bigram_probs[trigram[1:]]
            p += self.lambda3 * self.trigram_probs[trigram]
            
            # N_c - the count of things we've seen c times
#         else:
#             if trigram in train_trigram_counter:
#                 p = self.lambda1 * self.unigram_probs[trigram[-1]]
#                 p += self.lambda2 * self.bigram_probs[trigram[1:]]
#                 p += self.lambda3 * self.trigram_probs[trigram]
#             elif trigram[1:] in train_bigram_counter:
#                 p = (self.lambda1 * alpha1 + self.lambda2) * self.bigram_probs[trigram[1:]]
#                 p += self.lambda3 * self.trigram_probs[trigram]
#             elif trigram[-1] in train_unigram_counter:
#                 p = (self.lambda1 * alpha2 + self.lambda2 * 0.4 + self.lambda3) * self.trigram_probs[trigram]
#             else:
#                 return         
        return math.log(p)
            
    def test(self):
        perplexity = 0             
        n_trigrams = sum(test_trigram_counter.values())
        for trigram, count in test_trigram_counter.items():
            perplexity += count * self.logprob(trigram)
        perplexity = math.pow(2, -1*(perplexity/n_trigrams))     
        
        return perplexity

In [107]:
lm = LanguageModel()
lm.fit()
lm.test()

44.11243776960517